# Overview

This notebook shows you how to put Landsat data into an AWS deployment of the Open Data Cube.

## Credentials

We use the Boto3 library to work with AWS. Your credentials should be saved to `~/.aws/config`, the file should looks something like this:

```
[default]
aws_access_key_id = ABC123
aws_secret_access_key = /Z+ABC123
region = us-east-1
```

In [ ]:
import boto3
import tarfile
import io
import glob

## 2. Saving a Scene

The source data is decompressed, individual files are put to S3. We assume source input is available somewhere on a local filesystem. This function is used to expand a file and put each item into an S3 bucket.

In [ ]:
bucket = s3.Bucket('test-odc-bucket')

In [27]:
def expand_and_put(path):
    """"""
    with tarfile.open(path) as tar:
        print("processing: {}".format(path))
        for name in tar.getnames():
            print("saving: {}".format(name))
            data = tar.extractfile(name).read()
            bucket.put_object(Key=name, Body=data)

In [ ]:
base_path = '/espa-storage/downloads/lcmap/sites/washington/'

In [ ]:
for path in glob.glob(base_path + 'H05V02-*/LT05*1993*.tar.gz'):
    expand_and_put(path)
for path in glob.glob(base_path + 'H05V02-*/LT05*1994*.tar.gz'):
    expand_and_put(path)
for path in glob.glob(base_path + 'H05V02-*/LT05*1995*.tar.gz'):
    expand_and_put(path)
for path in glob.glob(base_path + 'H05V02-*/LT05*1996*.tar.gz'):
    expand_and_put(path)
for path in glob.glob(base_path + 'H05V02-*/LT05*1997*.tar.gz'):
    expand_and_put(path)
for path in glob.glob(base_path + 'H05V02-*/LT05*1998*.tar.gz'):
    expand_and_put(path)
for path in glob.glob(base_path + 'H05V02-*/LT05*1999*.tar.gz'):
    expand_and_put(path)

## 3. Build a Dataset YAML

TBD.

## 4. Build an Index

TBD.